In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
import NBInclude: @nbinclude
@nbinclude("Data.ipynb");

In [ ]:
const MIN_ITEMS = Dict("streaming" => 5, "test" => 5)
const TEST_USER_FRAC = Dict("streaming" => 0.1, "test" => 1.0)
const MAX_OUPUT_DAYS = Dict("streaming" => 7, "test" => 0)
const MAX_OUPUT_ITEMS = Dict("streaming" => 5, "test" => 5);
const OUTPUT_NEWEST = Dict("streaming" => true, "test" => false);

In [ ]:
dataset = ""

In [ ]:
function save_dataset(dataset)
    if !ispath(get_data_path("$(dataset)_data"))
        return
    end
    seed_rng!("ImportDatasets/ImportLists/$(titlecase(dataset))Data")
    dfs = load_datasets(dataset)
    train_dfs, test_dfs = split_by_user!(dfs, TEST_USER_FRAC[dataset])
    drop_sparse_users!(train_dfs, MIN_ITEMS[dataset])
    ts_cutoff =
        get_max_valid_ts("streaming") -
        (24 * 60 * 60 * MAX_OUPUT_DAYS[dataset]) / (MAX_TS - MIN_TS)
    test_input_dfs, test_output_dfs = create_splits!(
        test_dfs,
        ts_cutoff,
        MAX_OUPUT_ITEMS[dataset],
        OUTPUT_NEWEST[dataset],
    )
    filter_input!(test_input_dfs, test_output_dfs)
    relabel_userids!(train_dfs, test_input_dfs, test_output_dfs, dataset)
    save_dataset(train_dfs, test_input_dfs, test_output_dfs, dataset)
end;

In [ ]:
save_dataset(dataset)